# Using FISSA with Suite2p

Suite2P is blind source separation toolbox for cell detection and signal extraction. 

Here we illustrate how to use Suite2p to detect cell locations, and then use FISSA to remove neuropil signals from the ROI signals.

For more information about the Suite2p toolbox see: https://github.com/MouseLand/suite2p

Pachitariu, M., Stringer, C., Dipoppa, M., Schröder, S., Rossi, L. F., Dalgleish, H., Carandini, M. & Harris, K. D. (2017). Suite2p: beyond 10,000 neurons with standard two-photon microscopy. bioRxiv:[061507](https://www.biorxiv.org/content/10.1101/061507v2); doi:[10.1101/061507](https://doi.org/10.1101/061507).

The Suite2P parts of this tutorial are based on their [Jupyter notebook example](https://github.com/MouseLand/suite2p/blob/master/jupyter/run_pipeline_tiffs_or_batch.ipynb).

Note that the below results are not representative of either Suite2P or FISSA performance, as we are using a very small example dataset.

## Imports

In [ ]:
# Plotting toolbox
import holoviews as hv
import numpy as np
%load_ext holoviews.ipython

# FISSA
import fissa

# Suite2P
import suite2p.run_s2p

## Run Suite2p

In [ ]:
# Set your options for running
ops = suite2p.run_s2p.default_ops()  # populates ops with the default options

# Provide an h5 path in 'h5py' or a tiff path in 'data_path'
# db overwrites any ops (allows for experiment specific settings)
db = {
    'h5py': [],  # a single h5 file path
    'h5py_key': 'data',
    'look_one_level_down': False,  # whether to look in ALL subfolders when searching for tiffs
    'data_path': ['exampleData/20150529'],  # a list of folders with tiffs 
                                            # (or folder of folders with tiffs if look_one_level_down is True,
                                            # or subfolders is not empty)
    'save_path0': './',  # save path                                      
    'subfolders': [],  # choose subfolders of 'data_path' to look in (optional)
    'fast_disk': './',  # string which specifies where the binary file will be stored (should be an SSD)
    'reg_tif': True,  # save the motion corrected tiffs
    'tau': 0.7,  # timescale of gcamp6f
    'fs': 1,  # sampling rate
    'spatial_scale': 4
}

# Run one experiment
opsEnd = suite2p.run_s2p.run_s2p(ops=ops, db=db)

## Load the relevant data from the analysis

In [ ]:
# Extract the motion corrected tiffs (make sure that the reg_tif option is set to true, see above)
images = './suite2p/plane0/reg_tif'

# Load the detected regions of interest
stat = np.load('./suite2p/plane0/stat.npy', allow_pickle=True)  # cell stats
ops = np.load('./suite2p/plane0/ops.npy', allow_pickle=True).item()
iscell = np.load('./suite2p/plane0/iscell.npy', allow_pickle=True)[:, 0] 

# Get image size
Lx = ops['Lx']
Ly = ops['Ly']

# Get the cell ids
ncells = len(stat)
cell_ids = np.arange(ncells)  # assign each cell an ID, starting from 0.
cell_ids = cell_ids[iscell==1]  # only take the ROIs that are actually cells.
num_rois = len(cell_ids) 

# Generate ROI masks in a format usable by FISSA (in this case, a list of masks)
ROIs = [np.zeros((Ly, Lx), dtype=bool) for n in range(num_rois)]
for i, n in enumerate(cell_ids):
    # i is the position in cell_ids, and n is the actual cell number
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    ROIs[i][ypix, xpix] = 1

## Run FISSA with the defined ROIs and data

In [ ]:
output_folder = 'fissa_suite2p_example'
exp = fissa.Experiment(images, [ROIs[:ncells]], output_folder)
exp.separate()

## Plot the resulting ROI signals

In [ ]:
%%opts Curve {+axiswise}


def plot_cell_regions(cell, plot_neuropil=False):
    '''
    Plot a single cell region, with holoviews.
    '''
    out = hv.Overlay()
    if plot_neuropil:
        # Plot the neuropil as well as the ROI
        numReg = len(exp.roi_polys[cell][0])
    else:
        # Just plot the ROI, not the neuropil
        numReg = 1
    for i in range(numReg):
        # number of parts in the current region
        numParts = len(exp.roi_polys[cell][0][i])
        for j in range(numParts):
            x = exp.roi_polys[cell][0][i][j][:, 1]
            y = exp.roi_polys[cell][0][i][j][:, 0]
            out *= hv.Curve(zip(x, y)).opts(color='w')
    return out


# Get plots for all detected regions
region_plots = {i : plot_cell_regions(i) for i in range(exp.nCell)}

# Get plots for raw extracts and neuropil removed
traces_plots = {
    i: hv.Curve(exp.raw[i][1][0,:], label='Suite2p') * hv.Curve(exp.result[i][1][0,:], label='FISSA')
    for i in range(exp.nCell)
}

# Get average image
avg_img = hv.Raster(opsEnd[0]['meanImg'])

# Render holoviews
avg_img * hv.HoloMap(region_plots, kdims=['Cell']) + hv.HoloMap(traces_plots, kdims=['Cell'])

Note that with the above settings for Suite2P it seems to have detected more small local axon signals, instead of cells. This can possibly be improved with manual curation and Suite2P setting changes, but as noted above these results should not be seen as indicative for either Suite2P or FISSA due to the small dataset size. Also note that the above Suite2P traces are done without Suite2P's own neuropil removal algorithm.